# Exploring MITRE ATT&CK Data as a Graph


In [ ]:
# Install dependencies if needed

%pip install pandas
%pip install neontology
%pip install ontolocy
%pip install networkx
%pip install ipydatagrid
%pip install ipysigma
%pip install yfiles_jupyter_graphs

In [ ]:
import os

import pandas as pd
import networkx as nx
from ipysigma import Sigma
from ipydatagrid import DataGrid

# proprietary graph visualization library
from yfiles_jupyter_graphs import GraphWidget

from neontology import init_neontology, GraphConnection, Neo4jConfig
from neontology.utils import auto_constrain_neo4j


from ontolocy import MitreAttackGroup, MitreAttackTechnique, MitreAttackTactic
from ontolocy.tools import MitreAttackParser

In [ ]:
# if running on google colab, enable custom widgets
try:
    import google.colab
    from google.colab import output

    output.enable_custom_widget_manager()

except:
    pass

In [ ]:
# Function Definitions


def display_graph_sigma(input):
    link_data = input.node_link_data
    nx_graph = nx.node_link_graph(link_data, edges="edges", key="__pp__", name="__pp__")
    return Sigma(nx_graph, node_label="__str__", node_color="LABEL")


def display_graph_yfiles(input):
    link_data = input.node_link_data

    nx_graph = nx.node_link_graph(link_data, edges="edges", key="__pp__", name="__pp__")

    gw = GraphWidget(graph=nx_graph)

    gw.node_label_mapping = "__str__"

    return gw


def display_df(input):
    return DataGrid(input)

In [ ]:
# Config

mitre_attack_url = "https://github.com/mitre-attack/attack-stix-data/raw/refs/heads/master/enterprise-attack/enterprise-attack-15.1.json"

# By default, the cell looks for environment variables, otherwise specify connection info here
neo4j_uri = os.getenv("NEO4J_URI", "neo4j://localhost:7687")
neo4j_username = os.getenv("NEO4J_USERNAME", "neo4j")
neo4j_password = os.getenv("NEO4J_PASSWORD", "<PASSWORD>")

graph_config = Neo4jConfig(
    uri=neo4j_uri, username=neo4j_username, password=neo4j_password
)

In [ ]:
# Initialise the connection to the database

init_neontology(graph_config)
gc = GraphConnection()

In [ ]:
# If we don't already have MITRE ATT&CK data in the graph, populate it
# this can take a few minutes - it's about 20,000 relationships

if not MitreAttackTechnique.match_nodes(limit=1):
    auto_constrain_neo4j()
    attack_parser = MitreAttackParser()
    attack_parser.parse_url(mitre_attack_url)

In [ ]:
# We can use Ontolocy classes directly to look at certain Nodes in the graph

tactics = MitreAttackTactic.match_nodes()

tactic_records = [x.model_dump() for x in tactics]

tactic_df = pd.DataFrame.from_records(tactic_records)

# Show the dataframe natively
tactic_df[["attack_id", "name", "description", "attack_shortname"]].head()

In [ ]:
# Display the dataframe more interactively
display_df(tactic_df)

In [ ]:
# We can visualize how techniques relate to tactics

tactics_techniques_cypher = "MATCH (ta:MitreAttackTactic)-[r:MITRE_TACTIC_INCLUDES_TECHNIQUE]->(te:MitreAttackTechnique) RETURN *"

tactics_techniques_results = gc.evaluate_query(tactics_techniques_cypher)

display_graph_yfiles(tactics_techniques_results)

In [ ]:
# Techniques have some properties which retrieve information from related nodes

technique = MitreAttackTechnique.match_nodes(limit=1)
technique[0].tactic_names

In [ ]:
# Explore APT groups and the links between them

apt_cypher = """
MATCH (apt:MitreAttackGroup)-[r]->(o)
WHERE apt.name CONTAINS 'APT' 
RETURN * LIMIT 400
"""

apt_results = gc.evaluate_query(apt_cypher)

In [ ]:
# Display with yfiles

display_graph_yfiles(apt_results)

In [ ]:
# Alternatively, uncomment below to display with Sigma

# display_graph_sigma(apt_results)